# Final Project 


In [1]:
import argparse
import json
import pprint
import requests
import sys
import urllib
import csv
import time
import datetime
import pandas as pd
import glob
from pandas.io.json import json_normalize

from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [2]:
CLIENT_ID = 'gBcvfFpabk3CnzmXKdYe3w'
CLIENT_SECRET = 'K9dIltpo1DH4hTrKy6c7YhxR2bwENElpzqTEaRz44QXxKnImd3cSOnYmhrucVHns'
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
TOKEN_PATH = '/oauth2/token'
GRANT_TYPE = 'client_credentials'
SEARCH_LIMIT = 50

In [3]:
def obtaintoken(host, path):
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    assert CLIENT_ID, "Please supply your client_id."
    assert CLIENT_SECRET, "Please supply your client_secret."
    data = urlencode({
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': GRANT_TYPE,
    })
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }
    response = requests.request('POST', url, data=data, headers=headers)
    token = response.json()['access_token']
    return token

token = obtaintoken(API_HOST, TOKEN_PATH)
token


'Z1C_ixTGccfADwGnwzG5yUE_tqYbJ3Vfj33NrA5Qi_LhXdj_A0idBPo8mP2dHU71s6HAkJuCJDxhH-BJ1WSDno-pyfmit-GNuVB8gPlu2kxjw-tgGZSO6VHnG4H_WHYx'

In [4]:
def request(host, path, token, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % token,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [5]:
def search(token, term, location):
    

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, token, url_params=url_params)

def get_business(token, business_id):
    
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, token)

def query_api(term, location):
   
#     token = obtaintoken(API_HOST, TOKEN_PATH)
    response = search(token, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(token, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)


In [8]:
response = search(token, 'dinner', 'Boston, MA')
response
with open('data.json', 'w') as outfile:
    json.dump(response, outfile)


Querying https://api.yelp.com/v3/businesses/search ...


{'businesses': [{'categories': [{'alias': 'italian', 'title': 'Italian'},
    {'alias': 'mediterranean', 'title': 'Mediterranean'}],
   'coordinates': {'latitude': 42.36454, 'longitude': -71.05349},
   'display_phone': '(617) 367-4348',
   'distance': 2364.994170686,
   'id': 'pomodoro-boston-2',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/JB1KVFXAhGgm-Un-ONZHUw/o.jpg',
   'is_closed': False,
   'location': {'address1': '351 Hanover St',
    'address2': '',
    'address3': '',
    'city': 'Boston',
    'country': 'US',
    'display_address': ['351 Hanover St', 'Boston, MA 02113'],
    'state': 'MA',
    'zip_code': '02113'},
   'name': 'Pomodoro',
   'phone': '+16173674348',
   'price': '$$',
   'rating': 4.5,
   'review_count': 180,
   'transactions': [],
   'url': 'https://www.yelp.com/biz/pomodoro-boston-2?adjust_creative=gBcvfFpabk3CnzmXKdYe3w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=gBcvfFpabk3CnzmXKdYe3w'},
  {'categories': [{'alias': 'ne

In [30]:
# response = search(token, 'dinner', 'Boston, MA')
# response
# with open('data.json', 'w') as outfile:
#     json.dump(response, outfile)


Querying https://api.yelp.com/v3/businesses/search ...


In [6]:
d = datetime.datetime(2017,4,24,1,0,0)
unixtime = time.mktime(d.timetuple())
unixtime=int(unixtime)

In [7]:
def findallresult():
    page = 0
    url_params = {
                'term': '',
                'location': 'Boston, MA',
                'limit': SEARCH_LIMIT,
                'open_at':unixtime,               # Unix Timestamp
                'offset':0
            }
    response1 = request(API_HOST, SEARCH_PATH, token, url_params=url_params)
    with open('data/midnightsnack/page0.json', 'a') as outfile:
        json.dump(response1, outfile)
    
    
    
    
    
    
    while (page < (response1['total']/50)):
#    print 'The count is:', count
        page = page + 1
        url_params = {
                'term': '',
                'location': 'Boston, MA',
                'limit': SEARCH_LIMIT,
                'open_at':unixtime,               # Unix Timestamp
                'offset':50*page

            }

        response = request(API_HOST, SEARCH_PATH, token, url_params=url_params)
        with open('data/midnightsnack/page'+ str(page) +'.json', 'a') as outfile:
            json.dump(response, outfile)

In [53]:
findallresult()

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

In [57]:
# page=19
# while (page < (1239/50)):
# #    print 'The count is:', count
#         page = page + 1
#         url_params = {
#                 'term': '',
#                 'location': 'Boston, MA',
#                 'limit': SEARCH_LIMIT,
#                 'open_at':unixtime,               # Unix Timestamp
#                 'offset':50*page

#             }

#         response = request(API_HOST, SEARCH_PATH, token, url_params=url_params)
#         with open('data/midnightsnack/page'+ str(page) +'.json', 'a') as outfile:
#             json.dump(response, outfile)

In [8]:
pathList = []
for filepath in glob.glob('/Users/xiongyiming/Documents/INFO7374Python/Final Project/data/midnightsnack/*.json'):
    pathList.append(filepath)

In [11]:
with open('/Users/xiongyiming/Documents/INFO7374Python/Final Project/data/midnightsnack/page1.json') as data_file:    
    datajson = json.load(data_file)
    dfsu = json_normalize(datajson['businesses'])
    
dfsu.columns

Index(['categories', 'coordinates.latitude', 'coordinates.longitude',
       'display_phone', 'distance', 'id', 'image_url', 'is_closed',
       'location.address1', 'location.address2', 'location.address3',
       'location.city', 'location.country', 'location.display_address',
       'location.state', 'location.zip_code', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')

In [13]:
df = pd.DataFrame()
for file in pathList:
    with open(file) as data_file:    
        datajson = json.load(data_file)
        if 'businesses' in datajson:
            df1 = json_normalize(datajson['businesses'])
            df1 = df1.reindex(columns=dfsu.columns)
            df2 = df1[['name','distance', 'price', 'rating', 'review_count', 'location.display_address']]
            df = pd.concat([df,df2])

In [14]:
df

,name,distance,price,rating,review_count,location.display_address
0,Biddy Early's,1642.704354,$,4.5,193,"[141 Pearl St, Boston, MA 02110]"
1,Yvonne's,1172.085380,$$$,4.0,445,"[2 Winter Pl, Boston, MA 02108]"
2,Eastern Standard Kitchen and Drinks,2036.705065,$$$,4.0,1554,"[528 Commonwealth Ave, Boston, MA 02215]"
3,Lolita Cocina & Tequila Bar,598.312089,$$,4.0,1264,"[271 Dartmouth St, Boston, MA 02116]"
4,Anchovies,679.036390,$$,4.0,319,"[433 Columbus Ave, Boston, MA 02116]"
5,Franklin Café,578.808787,$$,4.0,479,"[278 Shawmut Ave, Boston, MA 02118]"
6,Corner Tavern,1520.979992,$$,4.0,214,"[421 Marlborough St, Boston, MA 02115]"
7,Croke Park/Whitey's,1900.952891,$,4.5,81,"[268 W Broadway, Boston, MA 02127]"
8,Moonshine 152,1370.297005,$$,4.0,182,"[152 Dorchester Ave, Boston, MA 02127]"
9,The Parish Cafe,443.523241,$$,4.0,1181,"[361 Boylston St, Boston, MA 02116]"


In [90]:
df.to_csv('output_path.csv',index=False,header=False)

In [96]:
len(df['name'].unique())

50